In [ ]:
# Put your name in this cell

In [1]:
# In this cell, import pandas as "pd".

http://pandas.pydata.org/pandas-docs/stable/io.html

In [2]:
# In this cell, read in the out_SP600.csv file (download from Blackboard) and assign to a dataframe "df".
# See above link for details on how to do this.

In [ ]:
# Execute this cell.
df.head(3)  # We see the data starts on 31-Dec-2012.

In [ ]:
# Execute this cell.
df.tail(3)   # We see the data ends on 31-Dec-2015, so we have 3 years of data.

In [ ]:
# Execute this cell.
type(df['Date'][0])  # Did you read in your dates as strings or as dates?

In [3]:
# In this cell, read in the out_SP600.csv file and assign to the dataframe "df",
# but this time take care to set the parameter that tells .read_csv() to read in dates as dates not strings.
# See above link for details on how to do this.

In [ ]:
# Execute this cell.
type(df['Date'][0])  # This should be a pandas Timestamp type.

In [ ]:
# Execute this cell.
df.head(3)  # Notice that our "Date" column is read in as just another column, rather than as an index.

In [4]:
# In this cell, read in the out_SP600.csv file and assign to the dataframe "df",
# but this time take care to set the parameter that tells .read_csv() to read in
# the "Date" column as an index.  See above link for details on how to do this.

In [ ]:
# Execute this cell.
df.head(3)  # You should now see your "Date" column as an index, rather than just another column.

In [ ]:
# Execute this cell.
df.shape    # You should see that we have 757 rows (dates) and 574 columns (stocks).
            # Having 2-dimensional data like this is often called "panel data".

In [5]:
# In this cell, select the "AAON" column and assign to "aaon".

In [ ]:
# Execute this cell.
type(aaon)   # This should be a pandas series.

In [ ]:
# Execute this cell.
import matplotlib.pyplot as plt

In [ ]:
# Execute this cell.
aaon.plot()  # This should display a plot of AAON's stock price immediately below.

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.pct_change.html

In [6]:
# In this cell take the percentage change of "aaon" series and assign it to a new series named "aaon_rtn".
# See above link for details on how to do this.

In [ ]:
# Execute this cell.
aaon_rtn.head()  # Notice that the first value is NaN, because we couldn't calculate the return for the first date.

In [ ]:
# Execute this cell.
aaon_rtn.describe()

In [7]:
# In this cell take the percentage change of "df" Dataframe and assign it to a new Dateframe named "df_rtn".

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.shift.html

In [8]:
# In this cell, take "df_rtn" and shift it by one day, so that today's value shows YESTERDAY's return,
# and assign the result to df_rtn_prev".  See above link for details on how to do this.

In [ ]:
# Execute this cell.
df_rtn_prev.head(7)  # The first two rows should have NaN, and Jan 3 should have Jan 2's return.

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.rolling.html

In [9]:
# In this cell, take "df_rtn_prev", and use the .rolling() and .sum() commands to get a rolling
# sum of prior (including the current date) 5 days.  Assign the result to "df_5d_rolling".
# See above link for details on how to do this.

In [ ]:
# Execute this cell.
df_5d_rolling.head(7) # The first six rows should have Nan, and Jan 9 should be the sum of the entries
# Jan 3, 4, 7, 8, 9 of df_rtn_prev, which is actually the sum of the entries Jan 2, 3, 4, 7, 8 of df_rtn.

In [ ]:
# Execute this cell.
df_5d_rolling_trimmed = df_5d_rolling.dropna()  # It is convenient to drop the rows with missing data.
df_5d_rolling_trimmed.head()

In [ ]:
# However, it might be convenient to modify a DataFrame "in place",
# rather than modifying a copy and assigning it to a new DataFrame (as was done in the previous cell).
# Execute this cell.
df_5d_rolling.dropna(inplace=True)
df_5d_rolling.head()

In [ ]:
# Execute this cell
mr_sig_raw = - df_5d_rolling
# The idea use the return over the prior 5 days as a trading signal.
# Our hypothesis is that we should have short-term mean reversion ("mr") of stock returns;
# that is why we apply a negative sign to our signal.  We short stocks that (relatively speaking)
# have gone up a lot in the previous 5 days, and we buy the stocks that have gone down.

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.mean.html

In [10]:
# In this cell, assign the cross sectional mean return to a pandas time series called "mr_sig_mean".
# See above link for details on how to do this.

In [ ]:
# Execute this cell.
mr_sig_mean.tail(3)

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.std.html

In [11]:
# In this cell, assign the cross sectional standard deviation to a pandas time series called "mr_sig_std".
# See above link for details on how to do this.

In [ ]:
# Execute this cell.
mr_sig_std.tail(3)

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sub.html    
https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.div.html

In [12]:
# It is convenient (but does not affect strategy performance) to normalize our signals to have mean 0, variance 1.
# So try to do something like mr_sig_norm = (mr_sig_raw - mr_sig_mean) / mr_sig_std (BUT THIS syntax won't work!)
# See above links for details of how to do this.

In [ ]:
# Execute this cell.
mr_sig_norm.tail(3)  # You should get panel of standard deviations.

In [ ]:
# We want to do something similar to get a momentum signal.  We want to consider the last year of
# returns (we'll use 252 trading days), but excluding the most recent month (21 prior trading days + today).
# Execute this cell.
mom_sig_raw = df_rtn.shift(22).rolling(231).sum().dropna()
mom_sig_raw.head(3)

In [13]:
# In this cell put in the various commands to normalize the momentum signal to have mean 0 and variance 1,
# and assign it to a DataFrame called "mom_sig_norm".

In [ ]:
# Execute this cell.
mom_sig_norm.tail(3)  # You should get panel of standard deviations.

In [ ]:
# Execute this cell.
mom_rtn = (mom_sig_norm/(574-1) * df_rtn).dropna().sum(axis=1)
# What we have done used mom_sig_norm / (N-1), with N=574 to use as a position size in dollars.
# I'll explain in lecture the connection between this and the Fama-Macbeth procedure for 1 factor.
# Here "mom_rtn" is a return of the momentum strategy in dollars, not percent.
# It is IMPORTANT to drop the NaN values BEFORE doing the cross sectional sum; otherwise
# you will get a lot of spurious zeros that won't be dropped.

In [ ]:
# Execute this cell.
mom_rtn_mean = mom_rtn.mean() * 252     # We express our mean return on an annual basis.
mom_rtn_std = mom_rtn.std() * 252**0.5  # We express our standard deviation on an annual basis.
mom_SR = mom_rtn_mean / mom_rtn_std     # This is the annual Sharpe ratio of our strategy.
print(mom_rtn_mean, mom_rtn_std, mom_SR)  # You should get a Sharpe ratio of +0.83

In [ ]:
# Execute this cell.
mr_rtn = (mr_sig_norm/(574-1) * df_rtn).truncate(before='2014-01-02').sum(axis=1)
# We truncate the time before Jan 2014 so that both the momentum series and the mean
# reversion series have the same date range (2 years of data).

In [14]:
# In this cell, put in the various commands to calculate annualized mean, standard deviation and
# Sharpe ratio of the mean reversion strategy.  You should get a Sharpe ratio of +0.87

In [ ]:
# Execute this cell.
both_rtn = 0.5*mom_rtn + 0.5*mr_rtn  # This is the return of a 50/50 portfolio of both strategies.

In [ ]:
# In this cell, put in the various commands to calculate annualized mean, standard deviation and
# Sharpe ratio of the 50/50 "both" strategy.  Did you expect a lower or higher Sharpe ratio?

In [ ]:
# Execute this cell.
plt.figure()  # This tells Python to start a new figure.
mom_rtn.cumsum().plot()
mr_rtn.cumsum().plot()
both_rtn.cumsum().plot()  # All three plots you be plotted on the same figure below.

In [ ]:
# Actually it make more sense to visually compare plots of strategies when they have been normalized to have
# the same volatility.  That way the highest Sharpe ratio strategy will also be shown to have the
# highest return.  Here we plot each strategy normalized to have 15% volatility (similar to S&P 500 index volatility).
# Execute this cell.
plt.figure()
(mom_rtn.cumsum()/mom_rtn_std*0.15).plot()
(mr_rtn.cumsum()/mr_rtn_std*0.15).plot()
(both_rtn.cumsum()/both_rtn_std*0.15).plot()

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html

In [15]:
# In this cell, create a single Dataframe "df_factors" made from mom_rtn and mr_rtn,
# and assign column names "mom" and "mr" respectively.
# See above link for details on how to do this.

In [ ]:
# Execute this cell.
df_factors.head(3)

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.cov.html

In [16]:
# In this cell, display the covariance matrix of df_factors.

https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.corr.html

In [ ]:
# In this cell, display the correlation matrix of df_factors.

Please think about this:

Given the returns, variances and covariances, how would you calculate the weighting of the two strategies to get the maximal Sharpe ratio?